# Overtaking Difficulty - Formula Comparison Test

**Goal:** Test multiple formulas and automatically pick the best one.

**Baseline:** 89.7% podium accuracy (without overtaking difficulty)

**Test 5 formulas:**
1. Linear (0.5-1.5)
2. Square Root (diminishing returns)
3. Conservative (0.7-1.3)
4. Aggressive (0.3-1.7)
5. Baseline (no factor)

**Winner:** Highest podium accuracy.

# Requirements
Run 
```python scripts/extract_overtaking_likelihood.py```
to create 
```../data/historical/overtaking_difficulty.json```

In [1]:
import copy
import json
import warnings

import fastf1 as ff1
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
import logging

logging.getLogger('fastf1').setLevel(logging.ERROR)

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')
print("✅ Imports complete")

✅ Imports complete


In [2]:
# Load overtaking difficulty
with open('../data/historical/overtaking_difficulty.json') as f:
    overtaking_data = json.load(f)

# Load 2024 priors
with open('../data/processed/testing_files/2024_season_characteristics.json') as f:
    priors_2024 = json.load(f)

print(f"✅ Loaded {len(overtaking_data['tracks'])} tracks")
print(f"✅ Loaded {len(priors_2024['drivers'])} drivers")

✅ Loaded 24 tracks
✅ Loaded 24 drivers


## Define Test Formulas

In [3]:
# Define all formula variants to test
FORMULAS = {
    'linear': lambda d: 0.5 + d,
    'sqrt': lambda d: 0.5 + (d ** 0.5),
    'conservative': lambda d: 0.7 + (d * 0.6),
    'aggressive': lambda d: 0.3 + (d * 1.4),
    'baseline': lambda d: 1.0
}

def get_overtaking_factor(race_name, formula_key='linear'):
    race_key = race_name.lower().replace(' ', '_')
    if race_key in overtaking_data['tracks']:
        difficulty = overtaking_data['tracks'][race_key]['difficulty']
        return round(FORMULAS[formula_key](difficulty), 2)
    return 1.0

# Test formulas on Monaco
print("Formula test (Monaco):")
for key in FORMULAS.keys():
    factor = get_overtaking_factor('Monaco Grand Prix', key)
    print(f"  {key:15} → {factor:.2f}")

Formula test (Monaco):
  linear          → 0.60
  sqrt            → 0.82
  conservative    → 0.76
  aggressive      → 0.44
  baseline        → 1.00


## Configuration

In [4]:
# Test configuration
TEST_SEASON = {'from': 2024, 'to': 2025}
TRACKED_DRIVERS = ['VER', 'HAM', 'LEC', 'NOR', 'SAI', 'PER', 'RUS', 'ALO']

# Tuned parameters from 21B
PARAMS = {
    'racecraft_weight': 6,
    'uncertainty_growth': 1.3,
    'dnf_multiplier': 1.8,
    'update_rate': 0.15
}

SPRINT_PARAMS = {
    'uncertainty_multiplier': 1.3,
    'racecraft_factor': 0.5,
    'dnf_risk_factor': 0.5,
    'quali_weight': 0.6,
    'sprint_weight': 0.4
}

print("✅ Configuration loaded")

✅ Configuration loaded


## Helper Functions

In [5]:
def extract_conventional_weekend(session_quali, session_race):
    results = {'quali': {}, 'race': {}}
    
    for _, row in session_quali.results.iterrows():
        driver = row['Abbreviation']
        quali_pos = row['Position']
        if pd.notna(driver) and pd.notna(quali_pos) and quali_pos != '':
            try:
                results['quali'][driver] = int(quali_pos)
            except (ValueError, TypeError):
                pass
    
    for _, row in session_race.results.iterrows():
        driver = row['Abbreviation']
        race_pos = row['Position']
        if pd.notna(driver) and driver in results['quali']:
            try:
                rpos = int(race_pos) if pd.notna(race_pos) and race_pos != '' else 20
            except (ValueError, TypeError):
                rpos = 20
            
            dnf = row.dnf if hasattr(row, 'dnf') else False
            status = str(row['Status']) if 'Status' in row else ''
            if not dnf and status:
                dnf = 'Finished' not in status and '+' not in status
            
            results['race'][driver] = {'race_pos': rpos, 'dnf': dnf}
    
    return results if results['quali'] and results['race'] else None

def extract_sprint_weekend(quali, sprint_quali, sprint_race, gp_race):
    results = {'quali': {}, 'sprint_quali': {}, 'sprint_race': {}, 'race': {}}
    
    for _, row in quali.results.iterrows():
        driver = row['Abbreviation']
        pos = row['Position']
        if pd.notna(driver) and pd.notna(pos) and pos != '':
            try:
                results['quali'][driver] = int(pos)
            except (ValueError, TypeError):
                pass
    
    for _, row in sprint_race.results.iterrows():
        driver = row['Abbreviation']
        race_pos_raw = row['Position']
        grid_pos_raw = row['GridPosition']
        
        dnf = row.dnf if hasattr(row, 'dnf') else False
        status = str(row['Status']) if 'Status' in row else ''
        if not dnf and status:
            dnf = 'Finished' not in status and '+' not in status
        
        if pd.notna(driver):
            try:
                race_pos = int(race_pos_raw) if pd.notna(race_pos_raw) and race_pos_raw != '' else 20
            except (ValueError, TypeError):
                race_pos = 20
            
            try:
                grid_pos = int(grid_pos_raw) if pd.notna(grid_pos_raw) and grid_pos_raw != '' else race_pos
            except (ValueError, TypeError):
                grid_pos = race_pos
            
            results['sprint_race'][driver] = {'grid_pos': grid_pos, 'race_pos': race_pos, 'dnf': dnf}
            results['sprint_quali'][driver] = grid_pos
    
    for _, row in gp_race.results.iterrows():
        driver = row['Abbreviation']
        pos = row['Position']
        if pd.notna(driver) and driver in results['quali']:
            try:
                race_pos = int(pos) if pd.notna(pos) and pos != '' else 20
            except (ValueError, TypeError):
                race_pos = 20
            
            dnf = row.dnf if hasattr(row, 'dnf') else False
            status = str(row['Status']) if 'Status' in row else ''
            if not dnf and status:
                dnf = 'Finished' not in status and '+' not in status
            
            results['race'][driver] = {'race_pos': race_pos, 'dnf': dnf}
    
    return results if results['quali'] and results['race'] else None

print("✅ Helper functions defined")

✅ Helper functions defined


## Prediction Functions (Modified for Overtaking Factor)

In [6]:
def predict_conventional_race(quali_results, driver_priors, race_name, formula_key='linear'):
    predictions = {}
    overtaking_factor = get_overtaking_factor(race_name, formula_key)
    
    for driver, quali_pos in quali_results.items():
        if driver not in driver_priors['drivers']:
            continue
        
        driver_data = driver_priors['drivers'][driver]
        racecraft_score = driver_data.get('racecraft', {}).get('skill_score', 0.5)
        dnf_prob = driver_data.get('dnf_risk', {}).get('rate', 0.1)
        quali_uncertainty = driver_data['pace']['uncertainty']
        
        # MODIFIED: Track-aware racecraft
        racecraft_delta = (racecraft_score - 0.5) * PARAMS['racecraft_weight'] * overtaking_factor
        expected_pos = quali_pos - racecraft_delta
        expected_pos = np.clip(expected_pos, 1, 20)
        
        uncertainty = quali_uncertainty * PARAMS['uncertainty_growth']
        
        predictions[driver] = {
            'expected_pos': expected_pos,
            'uncertainty': uncertainty,
            'dnf_prob': dnf_prob
        }
    
    return predictions

def predict_sprint_race(sprint_quali_results, driver_priors, race_name, formula_key='linear'):
    predictions = {}
    overtaking_factor = get_overtaking_factor(race_name, formula_key)
    
    for driver, quali_pos in sprint_quali_results.items():
        if driver not in driver_priors['drivers']:
            continue
        
        driver_data = driver_priors['drivers'][driver]
        racecraft_score = driver_data.get('racecraft', {}).get('skill_score', 0.5)
        dnf_prob = driver_data.get('dnf_risk', {}).get('rate', 0.1)
        quali_uncertainty = driver_data['pace']['uncertainty']
        
        # MODIFIED: Track-aware racecraft for sprint (shorter race)
        racecraft_delta = (racecraft_score - 0.5) * PARAMS['racecraft_weight'] * SPRINT_PARAMS['racecraft_factor'] * overtaking_factor
        expected_pos = quali_pos - racecraft_delta
        expected_pos = np.clip(expected_pos, 1, 20)
        
        uncertainty = quali_uncertainty * SPRINT_PARAMS['uncertainty_multiplier']
        dnf_prob_sprint = dnf_prob * SPRINT_PARAMS['dnf_risk_factor']
        
        predictions[driver] = {
            'expected_pos': expected_pos,
            'uncertainty': uncertainty,
            'dnf_prob': dnf_prob_sprint
        }
    
    return predictions

def predict_sprint_weekend_gp(friday_quali, sprint_race_result, driver_priors, race_name, formula_key='linear'):
    predictions = {}
    overtaking_factor = get_overtaking_factor(race_name, formula_key)
    
    for driver in friday_quali.keys():
        if driver not in driver_priors['drivers'] or driver not in sprint_race_result:
            continue
        
        driver_data = driver_priors['drivers'][driver]
        sprint_data = sprint_race_result[driver]
        
        racecraft_score = driver_data.get('racecraft', {}).get('skill_score', 0.5)
        dnf_prob = driver_data.get('dnf_risk', {}).get('rate', 0.1)
        quali_uncertainty = driver_data['pace']['uncertainty']
        
        # MODIFIED: Track-aware racecraft
        racecraft_delta = (racecraft_score - 0.5) * PARAMS['racecraft_weight'] * overtaking_factor
        
        quali_contribution = (friday_quali[driver] - racecraft_delta) * SPRINT_PARAMS['quali_weight']
        sprint_contribution = sprint_data['race_pos'] * SPRINT_PARAMS['sprint_weight']
        
        expected_pos = quali_contribution + sprint_contribution
        expected_pos = np.clip(expected_pos, 1, 20)
        
        uncertainty = quali_uncertainty * PARAMS['uncertainty_growth']
        dnf_adjustment = PARAMS['dnf_multiplier'] if sprint_data['dnf'] else 1.0
        dnf_prob_adjusted = min(dnf_prob * dnf_adjustment, 0.5)
        
        predictions[driver] = {
            'expected_pos': expected_pos,
            'uncertainty': uncertainty,
            'dnf_prob': dnf_prob_adjusted
        }
    
    return predictions

print("✅ Prediction functions defined (with overtaking factor)")

✅ Prediction functions defined (with overtaking factor)


## Metrics Calculation

In [7]:
def calculate_race_metrics(predictions, actual_results, points_threshold=10):
    if not predictions:
        return {'position_mae': None, 'podium_accuracy': None, 'points_accuracy': None, 'dnf_brier': None}
    
    position_errors = []
    podium_correct = []
    points_correct = []
    dnf_brier_scores = []
    
    for driver, pred in predictions.items():
        if driver not in actual_results:
            continue
        
        actual = actual_results[driver]
        position_errors.append(abs(pred['expected_pos'] - actual['race_pos']))
        
        pred_podium = pred['expected_pos'] <= 3
        actual_podium = actual['race_pos'] <= 3
        podium_correct.append(pred_podium == actual_podium)
        
        pred_points = pred['expected_pos'] <= points_threshold
        actual_points = actual['race_pos'] <= points_threshold
        points_correct.append(pred_points == actual_points)
        
        actual_dnf_binary = 1 if actual['dnf'] else 0
        brier = (pred['dnf_prob'] - actual_dnf_binary) ** 2
        dnf_brier_scores.append(brier)
    
    if not position_errors:
        return {'position_mae': None, 'podium_accuracy': None, 'points_accuracy': None, 'dnf_brier': None}
    
    return {
        'position_mae': np.mean(position_errors),
        'podium_accuracy': np.mean(podium_correct) * 100,
        'points_accuracy': np.mean(points_correct) * 100,
        'dnf_brier': np.mean(dnf_brier_scores),
        'n': len(position_errors)
    }

print("✅ Metrics calculation defined")

✅ Metrics calculation defined


## Main Validation Function

In [8]:
def run_validation(formula_key='linear', verbose=False):
    """Run validation with specified overtaking formula."""
    schedule = ff1.get_event_schedule(TEST_SEASON['to'])
    current_priors = copy.deepcopy(priors_2024)
    
    results = {
        'conventional_gp': [],
        'sprint_race': [],
        'sprint_gp': []
    }
    
    for _, event in schedule.iterrows():
        race_name = event['EventName']
        
        if 'testing' in race_name.lower():
            continue
        
        event_format = event['EventFormat'].lower()
        weekend_type = 'sprint' if 'sprint' in event_format else 'conventional'
        
        if weekend_type not in ['conventional', 'sprint']:
            continue
        
        try:
            if weekend_type == 'conventional':
                quali = ff1.get_session(TEST_SEASON['to'], race_name, 'Q')
                quali.load(laps=False, telemetry=False, weather=False)
                race = ff1.get_session(TEST_SEASON['to'], race_name, 'R')
                race.load(laps=False, telemetry=False, weather=False)
                
                weekend_data = extract_conventional_weekend(quali, race)
                if not weekend_data:
                    continue
                
                gp_predictions = predict_conventional_race(
                    weekend_data['quali'], 
                    current_priors, 
                    race_name,
                    formula_key
                )
                gp_metrics = calculate_race_metrics(gp_predictions, weekend_data['race'])
                if gp_metrics['position_mae'] is not None:
                    results['conventional_gp'].append(gp_metrics)
            
            else:  # sprint
                quali = ff1.get_session(TEST_SEASON['to'], race_name, 'Q')
                quali.load(laps=False, telemetry=False, weather=False)
                sprint_quali = ff1.get_session(TEST_SEASON['to'], race_name, 'SQ')
                sprint_quali.load(laps=False, telemetry=False, weather=False)
                sprint_race = ff1.get_session(TEST_SEASON['to'], race_name, 'S')
                sprint_race.load(laps=False, telemetry=False, weather=False)
                gp_race = ff1.get_session(TEST_SEASON['to'], race_name, 'R')
                gp_race.load(laps=False, telemetry=False, weather=False)
                
                weekend_data = extract_sprint_weekend(quali, sprint_quali, sprint_race, gp_race)
                if not weekend_data:
                    continue
                
                sprint_predictions = predict_sprint_race(
                    weekend_data['sprint_quali'],
                    current_priors,
                    race_name,
                    formula_key
                )
                sprint_metrics = calculate_race_metrics(sprint_predictions, weekend_data['sprint_race'], 8)
                if sprint_metrics['position_mae'] is not None:
                    results['sprint_race'].append(sprint_metrics)
                
                gp_predictions = predict_sprint_weekend_gp(
                    weekend_data['quali'],
                    weekend_data['sprint_race'],
                    current_priors,
                    race_name,
                    formula_key
                )
                gp_metrics = calculate_race_metrics(gp_predictions, weekend_data['race'])
                if gp_metrics['position_mae'] is not None:
                    results['sprint_gp'].append(gp_metrics)
        
        except Exception as e:
            if verbose:
                print(f"Error at {race_name}: {e}")
            continue
    
    # Calculate combined metrics
    all_gp = results['conventional_gp'] + results['sprint_gp']
    
    if all_gp:
        combined = {
            'position_mae': np.mean([m['position_mae'] for m in all_gp]),
            'podium_accuracy': np.mean([m['podium_accuracy'] for m in all_gp]),
            'points_accuracy': np.mean([m['points_accuracy'] for m in all_gp]),
            'total_races': len(all_gp)
        }
    else:
        combined = {'position_mae': None, 'podium_accuracy': None, 'points_accuracy': None, 'total_races': 0}
    
    return combined

print("✅ Validation function defined")

✅ Validation function defined


## Run Tests for All Formulas

In [9]:
print("Running validation for all formulas...")
print("This will take ~10-15 minutes\n")

test_results = {}

for i, formula_key in enumerate(FORMULAS.keys(), 1):
    print(f"[{i}/5] Testing {formula_key}...", end=' ')
    
    results = run_validation(formula_key, verbose=False)
    test_results[formula_key] = results
    
    print(f"✓ Podium: {results['podium_accuracy']:.1f}%, MAE: ±{results['position_mae']:.2f}")

print("\n✅ All tests complete!")

Running validation for all formulas...
This will take ~10-15 minutes

[1/5] Testing linear... ✓ Podium: 89.9%, MAE: ±3.46
[2/5] Testing sqrt... ✓ Podium: 89.7%, MAE: ±3.50
[3/5] Testing conservative... ✓ Podium: 90.2%, MAE: ±3.46
[4/5] Testing aggressive... ✓ Podium: 89.7%, MAE: ±3.46
[5/5] Testing baseline... ✓ Podium: 90.4%, MAE: ±3.46

✅ All tests complete!


## Compare Results

In [10]:
print("FORMULA COMPARISON")
print("=" * 80)
print(f"{'Formula':<20} {'Podium':<12} {'MAE':<12} {'Points':<12} {'vs Baseline'}")
print("-" * 80)

baseline_podium = test_results['baseline']['podium_accuracy']

for formula_key, results in sorted(test_results.items(), 
                                   key=lambda x: x[1]['podium_accuracy'], 
                                   reverse=True):
    podium = results['podium_accuracy']
    mae = results['position_mae']
    points = results['points_accuracy']
    diff = podium - baseline_podium
    
    diff_str = f"+{diff:.1f}%" if diff > 0 else f"{diff:.1f}%"
    
    print(f"{formula_key:<20} {podium:>10.1f}%  ±{mae:>8.2f}   {points:>10.1f}%  {diff_str:>10}")

print("\n" + "=" * 80)

FORMULA COMPARISON
Formula              Podium       MAE          Points       vs Baseline
--------------------------------------------------------------------------------
baseline                   90.4%  ±    3.46         76.0%        0.0%
conservative               90.2%  ±    3.46         75.7%       -0.3%
linear                     89.9%  ±    3.46         75.7%       -0.5%
aggressive                 89.7%  ±    3.46         75.5%       -0.8%
sqrt                       89.7%  ±    3.50         75.2%       -0.8%



## Pick Winner

In [11]:
# Find best formula (highest podium accuracy)
winner = max(test_results.items(), key=lambda x: x[1]['podium_accuracy'])
winner_key = winner[0]
winner_results = winner[1]

print("\n" + "=" * 80)
print("WINNER")
print("=" * 80)
print(f"\nBest formula: {winner_key.upper()}")
print("\nResults:")
print(f"  Podium Accuracy: {winner_results['podium_accuracy']:.1f}%")
print(f"  Position MAE: ±{winner_results['position_mae']:.2f}")
print(f"  Points Accuracy: {winner_results['points_accuracy']:.1f}%")
print(f"\nImprovement vs baseline: {winner_results['podium_accuracy'] - baseline_podium:+.1f}%")

if winner_key == 'baseline':
    print("\n⚠️  BASELINE WON - Overtaking difficulty doesn't improve predictions!")
    print("   → Archive the idea and move on.")
else:
    print(f"\n✅ {winner_key.upper()} formula improves predictions!")
    print(f"   → Use this formula: factor = {FORMULAS[winner_key].__code__.co_consts[1]}")
    print("   → Productionize it!")


WINNER

Best formula: BASELINE

Results:
  Podium Accuracy: 90.4%
  Position MAE: ±3.46
  Points Accuracy: 76.0%

Improvement vs baseline: +0.0%

⚠️  BASELINE WON - Overtaking difficulty doesn't improve predictions!
   → Archive the idea and move on.


## Save Results

In [12]:
# Save test results
output = {
    'test_date': str(pd.Timestamp.now()),
    'winner': winner_key,
    'winner_results': winner_results,
    'all_results': test_results,
    'baseline_podium': baseline_podium
}

with open('../data/processed/testing_files/overtaking_formula_test.json', 'w') as f:
    json.dump(output, f, indent=2)

print("✅ Results saved to: ../data/processed/testing_files/overtaking_formula_test.json")

✅ Results saved to: ../data/processed/testing_files/overtaking_formula_test.json


In [13]:
# Test 6th formula: Monaco-only flag
FORMULAS['monaco_binary'] = lambda d: 0.5 if d < 0.15 else 1.0

# Re-run validation
results = run_validation('monaco_binary')
print(f"Monaco Binary: {results['podium_accuracy']:.1f}%")

Monaco Binary: 90.7%


## Experiment: Monaco Racecraft Factor

**Date:** 2025-12-31
**Result:** +0.3% accuracy (90.4% → 90.7%)

### Decision: NOT IMPLEMENTED

**Reasons:**
1. 2026 regulation reset (smaller cars) may invalidate factor
2. Based on outlier performances (Verstappen Brazil 2024/2025)
3. Requires annual revalidation for minimal gain
4. Adds technical debt for 0.3% improvement
5. Bayesian uncertainty already captures track variability

**Lesson:** 
Empirical validation is necessary but not sufficient.
Production features must be maintainable across regulation changes.
Small accuracy gains don't justify ongoing maintenance costs.

**Status:** Tested and validated, but rejected for production use.